In [0]:
import os
import matplotlib.pyplot as plt
import numpy as np
from shutil import copyfile
import torch
import torch.nn as nn
import torchvision.datasets as datasets
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader

In [0]:
#download complete patches dataset http://places2.csail.mit.edu/download.html
!wget http://data.csail.mit.edu/places/places365/places365standard_easyformat.tar


--2019-09-18 13:20:09--  http://data.csail.mit.edu/places/places365/places365standard_easyformat.tar
Resolving data.csail.mit.edu (data.csail.mit.edu)... 128.52.129.40
Connecting to data.csail.mit.edu (data.csail.mit.edu)|128.52.129.40|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26686689280 (25G) [application/octet-stream]
Saving to: ‘places365standard_easyformat.tar’

365standard_easyfor   1%[                    ] 360.84M  27.6MB/s    eta 16m 6s 

In [0]:
#unpack file replace filename
!tar -xvf places365standard_easyformat.tar

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
def is_rgb(folder, filename):
    img = plt.imread(os.path.join(folder, filename))
    if img.shape != (256, 256, 3) or np.var(img,axis=2).max() < 100:
        return False
    else:
        return True

#not usable for big dataset
def copy_files(source, dest, indices=None):
    
    '''
    source/dest: string: source/destination folder path 
    indices: list [start, stop] or None: first and last image to be copied or if None all images in directory are copied
    '''
    if indices == None:
        start,stop = [0,len(os.listdir(source))]
    else:
        start,stop = indices
    list_path = dest + '-list.txt'
    
    if len(os.listdir(source)) < stop:
        print("stop index out of range, no files copied, maximum for stop in",source,"is",len(os.listdir(source)))
        return
    try:
        with open(list_path, 'r') as f:
            file_list = [line.rstrip('\n') for line in f]

    except FileNotFoundError:
        print("No file-list found, initializing new one.")
        file_list = []

    img_index = 0
    new_images = 0
    for filename in os.listdir(source):
        if img_index >= start and img_index < stop:
            if is_rgb(source, filename) and filename not in file_list:
                copyfile(os.path.join(source, filename), os.path.join(dest, filename))
                file_list.append(filename)
                new_images += 1
        img_index += 1

    with open(list_path, 'w') as f:
        for s in file_list:
            f.write(str(s) + '\n')
    print("Added",new_images,"out of",stop-start,"images")
    
#for big dataset
def copy_big_set(source, dest, train ,indices):
    '''
    source/dest: string: source/destination folder path 
    indices: list [start, stop] or None: first and last image to be copied or if None all images in directory are copied
    train: bool: copy elements of training (True) or validation set (False)
    '''
    if train:
        path_ending = 'train'
    else:
        path_ending = 'val'
     
    
    if indices == None:
        start,stop = [0,len(os.listdir(os.path.join(source, path_ending)))]
    else:
        start,stop = indices
    
    #get lists of files to copy from and allready copied filenames
    src_list_path = os.path.join(source, path_ending + '.txt')
    dest_list_path = os.path.join(dest, path_ending + '.txt')
    
    with open(src_list_path, 'r') as f:
        src_list = [line.rstrip('\n') for line in f]        
    try:
      with open(dest_list_path, 'r') as f:
          dest_list = [line.rstrip('\n') for line in f]
    except FileNotFoundError:
      dest_list = []
    
    
    img_index = 0
    new_images = 0
    for filename in src_list:
        if img_index >= start and img_index < stop:
            if is_rgb(source, filename) and filename not in dest_list:
                try:
                    copyfile(os.path.join(source, filename), os.path.join(dest, filename))
                    
                except FileNotFoundError:
                    subpath,_ = filename.rsplit('/',1)
                    os.mkdir(os.path.join(dest,subpath))
                    copyfile(os.path.join(source, filename), os.path.join(dest, filename))

                dest_list.append(filename)
                new_images += 1
        img_index += 1

    with open(dest_list_path, 'w') as f:
        for s in dest_list:
            f.write(str(s) + '\n')
    print("Added",new_images,"out of",stop-start,"images")

In [0]:
copy_big_set('places365_standard',"drive/My Drive/Uni/6. Semester/LOCAL/data/places-big",True,[90000,100000])

Added 0 out of 10000 images
